In [ ]:
!pip3 install accelerate safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

### Test model on its own

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "nvidia/Llama3-ChatQA-1.5-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

messages = [
    {"role": "user", "content": "what is the percentage change of the net income from Q4 FY23 to Q4 FY24?"}
]

document = """NVIDIA (NASDAQ: NVDA) today reported revenue for the fourth quarter ended January 28, 2024, of $22.1 billion, up 22% from the previous quarter and up 265% from a year ago.\nFor the quarter, GAAP earnings per diluted share was $4.93, up 33% from the previous quarter and up 765% from a year ago. Non-GAAP earnings per diluted share was $5.16, up 28% from the previous quarter and up 486% from a year ago.\nQ4 Fiscal 2024 Summary\nGAAP\n| $ in millions, except earnings per share | Q4 FY24 | Q3 FY24 | Q4 FY23 | Q/Q | Y/Y |\n| Revenue | $22,103 | $18,120 | $6,051 | Up 22% | Up 265% |\n| Gross margin | 76.0% | 74.0% | 63.3% | Up 2.0 pts | Up 12.7 pts |\n| Operating expenses | $3,176 | $2,983 | $2,576 | Up 6% | Up 23% |\n| Operating income | $13,615 | $10,417 | $1,257 | Up 31% | Up 983% |\n| Net income | $12,285 | $9,243 | $1,414 | Up 33% | Up 769% |\n| Diluted earnings per share | $4.93 | $3.71 | $0.57 | Up 33% | Up 765% |"""

def get_formatted_input(messages, context):
    system = "System: This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the context. The assistant should also indicate when the answer cannot be found in the context."
    instruction = "Please give a full and complete answer for the question."

    for item in messages:
        if item['role'] == "user":
            ## only apply this instruction for the first user turn
            item['content'] = instruction + " " + item['content']
            break

    conversation = '\n\n'.join(["User: " + item["content"] if item["role"] == "user" else "Assistant: " + item["content"] for item in messages]) + "\n\nAssistant:"
    formatted_input = system + "\n\n" + context + "\n\n" + conversation

    return formatted_input

formatted_input = get_formatted_input(messages, document)
tokenized_prompt = tokenizer(tokenizer.bos_token + formatted_input, return_tensors="pt").to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(input_ids=tokenized_prompt.input_ids, attention_mask=tokenized_prompt.attention_mask, max_new_tokens=128, eos_token_id=terminators)

response = outputs[0][tokenized_prompt.input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/6.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 The percentage change of the net income from Q4 FY23 to Q4 FY24 is calculated using the formula (($12,285 million - $1,414 million) / $1,414 million * 100), resulting in a 769% increase.


### Create a pipeline for the model

In [ ]:
from transformers import Pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers.pipelines import PIPELINE_REGISTRY
import torch
from torch.nn.functional import softmax
import numpy as np

class DocumentQuestionAnsweringPipeline(Pipeline):

    def _sanitize_parameters(self, **kwargs):
            return {}, {}, {}

    def preprocess(self, inputs):
      messages = [{"role": "user", "content": inputs}]
      # document = inputs["document"]
      document = ""
      system = "System: This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the context. The assistant should also indicate when the answer cannot be found in the context."
      instruction = "Please give a full and complete answer for the question."

      for item in messages:
          if item['role'] == "user":
              item['content'] = instruction + " " + item['content']
              break

      conversation = '\n\n'.join(["User: " + item["content"] if item["role"] == "user" else "Assistant: " + item["content"] for item in messages]) + "\n\nAssistant:"
      formatted_input = system + "\n\n" + document + "\n\n" + conversation

      return self.tokenizer(self.tokenizer.bos_token + formatted_input, return_tensors="pt").to(self.model.device)



    def _forward(self, model_inputs):
        terminators = [self.tokenizer.eos_token_id, self.tokenizer.convert_tokens_to_ids("<|eot_id|>")]

        outputs = self.model.generate(**model_inputs,
        max_new_tokens=128,
        eos_token_id=terminators,
        output_scores=True,
        return_dict_in_generate=True)

        return {"output": outputs, "input": model_inputs}

    def postprocess(self, model_outputs):
        output = model_outputs['output']
        input = model_outputs['input']
        # print("hereeee")

        sequences = output.sequences
        input_length = input['input_ids'].shape[-1]

        generated_sequence = sequences[0, input_length:]
        decoded_resp = self.tokenizer.decode(generated_sequence, skip_special_tokens=True)

        # Calculate confidence
        scores = output.scores
        log_probs = [np.log(softmax(score, dim=-1).max().item()) for score in scores]
        total_log_prob = np.sum(log_probs)
        confidence = np.exp(total_log_prob / len(log_probs))

        return {'guess': decoded_resp, 'confidence': confidence}




### Test model

In [ ]:
from transformers import pipeline

# Register pipeline
PIPELINE_REGISTRY.register_pipeline("document-qa", pipeline_class=DocumentQuestionAnsweringPipeline)

model_id="nvidia/Llama3-ChatQA-1.5-8B"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Initialize the pipeline
doc_qa_pipeline = pipeline("document-qa", model=model, tokenizer=tokenizer)

q = "This film was screen-played by two persons, one of them has the same name as a Anglican priest who died in 1932. It was nominated for four academy awards. In its sequel, Douglas Rain (May 9, 1928 - November 11, 2018) worked as the voice actor of an artificial intelligence character. What is the name of this film?"
response = doc_qa_pipeline(q)
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Push to the hub

In [ ]:
from pipeline import DocumentQuestionAnsweringPipeline
from transformers import pipeline
from transformers.pipelines import PIPELINE_REGISTRY
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

PIPELINE_REGISTRY.register_pipeline("document-qa", pipeline_class=DocumentQuestionAnsweringPipeline)

model_id="nvidia/Llama3-ChatQA-1.5-8B"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Initialize the pipeline
doc_qa_pipeline = pipeline("document-qa", model=model, tokenizer=tokenizer)
doc_qa_pipeline.push_to_hub("kitopang/llama3_generative_qa", safe_serialization=False)




pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/6.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

...

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kitopang/generative_qa/commit/d0311b218f523c07a45d8cbf54c97d989662b2fe', commit_message='Upload DocumentQuestionAnsweringPipeline', commit_description='', oid='d0311b218f523c07a45d8cbf54c97d989662b2fe', pr_url=None, pr_revision=None, pr_num=None)

### Try loading it from the hub

In [ ]:
from transformers import pipeline

test_pipe = pipeline("document-qa", model="kitopang/llama3_generative_qa", trust_remote_code=True)

config.json:   0%|          | 0.00/845 [00:00<?, ?B/s]

pipeline.py:   0%|          | 0.00/2.61k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/kitopang/llama3_generative_qa:
- pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
q = "This film was screen-played by two persons, one of them has the same name as a Anglican priest who died in 1932. It was nominated for four academy awards. In its sequel, Douglas Rain (May 9, 1928 - November 11, 2018) worked as the voice actor of an artificial intelligence character. What is the name of this film?"

print(test_pipe(q))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'guess': ' The film is called 2001: A Space Odyssey.', 'confidence': 0.7079861200674769}
